In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import subprocess as sp

In [ ]:
def read_dist(filename):
    with open(filename) as f:
        #w, h = [int(x) for x in next(f).split()] # read first line
        ys = []
        for line in f: # read rest of lines
            ys.append([int(x) for x in line.split()])
        return ys

In [ ]:
def get_compress_size(dataset, compressor="sz", nprocs=1):
    '''
    calls the libpressio container via runcmd wrapper
    returns the compressed size across the given commpressor and 
    process count extracted from json
    '''
    result = sp.run(f"./runcmd data_dist.py {nprocs} | jq '.{compressor}[].metrics[\"size:compressed_size\"]'",
          shell=True, check=True, capture_output=True, text=False)
    
    return [int(x) for x in str(result.stdout, encoding='utf-8').splitlines()]

In [ ]:
def plot_compress(ys):
    xs = [x for x in range(len(ys))]
    plt.plot(xs, ys)
    plt.show()
    # Make sure to close the plt object once done
    plt.close()

In [ ]:
def plot_hist(ys):
    npa = np.asarray(ys, dtype=np.float32)
    plt.hist(npa)
    plt.show()

In [ ]:
ys=get_compress_size(dataset="", compressor="zfp", nprocs="100")

In [ ]:
plot_compress(ys)

In [ ]:
plot_hist(ys)

In [ ]:
ys=read_dist('sz-100-slice')

In [ ]:
plot_compress(ys)

In [ ]:
plot_hist(ys)

In [ ]:
ys=read_dist('zfp-25-slice')

In [ ]:
plot_compress(ys)

In [ ]:
plot_hist(ys)

In [ ]:
ys=read_dist('sz-25-slice')

In [ ]:
plot_compress(ys)

In [ ]:
plot_hist(ys)